# Guild Wars 2 Trader: Same-day flip

This notebook identifies profitable same-day flips.

In [ ]:
import re
import math
import requests
import logging

import pandas as pd
import numpy as np
import plotly
import ipywidgets as widgets

from plotly.subplots import make_subplots
from dataclasses import dataclass
from operator import attrgetter
from enum import Enum
from typing import List, Self, Optional
from gw2tpdb import Gw2TpDb
from gw2tpdb.api.history import HistoryEntry
from urllib.parse import urlencode
from bs4 import BeautifulSoup
from coins import Coins
from item import Item
from gw2bltc import get_top_1000_sold_items
from tp_profit import profit
from IPython.display import Markdown, display

In [ ]:
stack_size = 250

In [ ]:
logging.basicConfig(
    format="%(asctime)s %(levelname)-8s [%(name)s] %(message)s",
    filename="trader.log",
    level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [ ]:
@dataclass
class DailyFlipReport():
    """TODO"""

    gw2bltc_url: str
    item_id: int
    item_name: str
    return_on_investment: float
    max_buy_count: float
    max_invest: Coins
    buy_price: Coins
    sell_price: Coins
    buy_volume: int
    sell_volume: int
    outlier_count: int

@dataclass
class Analysis():
    """TODO"""

    item: Item
    # TODO: Specify
    df: pd.DataFrame
    outlier_count: int

def sort_history_by_timestamp(entries: List[HistoryEntry], reverse: bool = False) -> List[HistoryEntry]:
    """Sort ENTRIES by timestamp field."""

    return sorted(entries, key=attrgetter("utc_timestamp"), reverse=reverse)

def history_to_pandas(entries: List[HistoryEntry]) -> pd.DataFrame:
    """Return n-dimensional numpy array for ENTRIES."""

    return pd.DataFrame(entries, columns=["id",
                                     "buy_delisted",
                                     "buy_listed",
                                     "buy_price_avg",
                                     "buy_price_max",
                                     "buy_price_min",
                                     "buy_price_stdev",
                                     "buy_quantity_avg",
                                     "buy_quantity_max",
                                     "buy_quantity_min",
                                     "buy_quantity_stdev",
                                     "buy_sold",
                                     "buy_value",
                                     "count",
                                     "sell_delisted",
                                     "sell_listed",
                                     "sell_price_avg",
                                     "sell_price_max",
                                     "sell_price_min",
                                     "sell_price_stdev",
                                     "sell_quantity_avg",
                                     "sell_quantity_max",
                                     "sell_quantity_min",
                                     "sell_quantity_stdev",
                                     "sell_sold",
                                     "sell_value",
                                     "utc_timestamp",])


def analyze_daily_flip(item: Item, entries: List[HistoryEntry], moving_average_window_size: int) -> Analysis:
    """TODO"""

    df = history_to_pandas(sort_history_by_timestamp(entries))

    df["buy_price_avg_-1stdev"] = df["buy_price_avg"] - (1 * df["buy_price_stdev"])
    df[f"buy_price_avg_-1stdev_{moving_average_window_size}d_ma"] = df["buy_price_avg_-1stdev"].rolling(moving_average_window_size).mean()
    df["sell_price_avg_+1stdev"] = df["sell_price_avg"] + (1 * df["sell_price_stdev"])
    df[f"sell_price_avg_+1stdev_{moving_average_window_size}d_ma"] = df["sell_price_avg_+1stdev"].rolling(moving_average_window_size).mean()

    df["sell_price_avg_pct_change"] = df["sell_price_avg"].pct_change(fill_method=None)
    sell_price_pct_change_point_75_quantile = df["sell_price_avg_pct_change"].quantile(0.75)
    sell_price_pct_change_point_25_quantile = df["sell_price_avg_pct_change"].quantile(0.25)
    sell_price_pct_change_interquartile_range = sell_price_pct_change_point_75_quantile - sell_price_pct_change_point_25_quantile
    df["outlier_sell_price_avg_pct_change"] = df["sell_price_avg_pct_change"].apply(math.fabs) > sell_price_pct_change_point_75_quantile + (1.5 * sell_price_pct_change_interquartile_range)

    df["buy_price_avg_pct_change"] = df["buy_price_avg"].pct_change(fill_method=None)
    buy_price_pct_change_point_75_quantile = df["buy_price_avg_pct_change"].quantile(0.75)
    buy_price_pct_change_point_25_quantile = df["buy_price_avg_pct_change"].quantile(0.25)
    buy_price_pct_change_interquartile_range = buy_price_pct_change_point_75_quantile - buy_price_pct_change_point_25_quantile
    df["outlier_buy_price_avg_pct_change"] = df["buy_price_avg_pct_change"].apply(math.fabs) > buy_price_pct_change_point_75_quantile + (1.5 * buy_price_pct_change_interquartile_range)

    sell_outlier_count = len([x for x in df["outlier_sell_price_avg_pct_change"][-1*moving_average_window_size:] if x])
    buy_outlier_count = len([x for x in df["outlier_buy_price_avg_pct_change"][-1*moving_average_window_size:] if x])
    """
    if (item.id == 36038):
        logger.debug(item)
        logger.debug(sell_price_pct_change_point_75_quantile + (1.5 * sell_price_pct_change_interquartile_range))
        logger.debug(df[[
            "utc_timestamp",
            #"sell_price_avg_pct_change",
            "outlier_sell_price_avg_pct_change",
            #"buy_price_avg_pct_change",
            "outlier_buy_price_avg_pct_change",
        ]])
    """

    """
    df["same_day_flip_profit"] = same_day_flip_profit(df,
                                    buy_price_column_name="buy_price_avg",
                                    sell_price_column_name="sell_price_avg")
    """
    df["same_day_flip_profit_1stdev"] = profit(df[f"buy_price_avg_-1stdev_{moving_average_window_size}d_ma"],
                                    df[f"sell_price_avg_+1stdev_{moving_average_window_size}d_ma"])
    """
    df["same_day_flip_profit_2stdev"] = same_day_flip_profit(df,
                                    buy_price_column_name="buy_price_avg_-2stdev",
                                    sell_price_column_name="sell_price_avg_+1stdev")
    """
    #df["same_day_flip_roi"] = (df["same_day_flip_profit"] + df["buy_price_avg"]) / df["buy_price_avg"]
    df["same_day_flip_1stdev_roi"] = (df["same_day_flip_profit_1stdev"] + df[f"buy_price_avg_-1stdev_{moving_average_window_size}d_ma"]) / df[f"buy_price_avg_-1stdev_{moving_average_window_size}d_ma"]
    #df["same_day_flip_2stdev_roi"] = (df["same_day_flip_profit_2stdev"] + df["buy_price_avg_-2stdev"]) / df["buy_price_avg_-2stdev"]

    # Moving averages
    df[f"buy_sold_{moving_average_window_size}d_ma"] = df["buy_sold"].rolling(moving_average_window_size).mean()
    #df["buy_value_30d_ma"] = df["buy_value"].rolling(30).mean()

    #df["sell_sold_7d_ma"] = df["sell_sold"].rolling(7).mean()
    #df["sell_sold_14d_ma"] = df["sell_sold"].rolling(14).mean()
    #df["sell_sold_30d_ma"] = df["sell_sold"].rolling(30).mean()
    df[f"sell_sold_{moving_average_window_size}d_ma"] = df["sell_sold"].rolling(moving_average_window_size).mean()

    #df["sell_value_{moving_average_window_size}d_ma"] = df["sell_value"].rolling(moving_average_window_size).mean()

    #df["same_day_flip_roi_30d_ma"] = df["same_day_flip_roi"].rolling(30).mean()
    #df["same_day_flip_1stdev_roi_7d_ma"] = df["same_day_flip_1stdev_roi"].rolling(7).mean()
    #df["same_day_flip_1stdev_roi_14d_ma"] = df["same_day_flip_1stdev_roi"].rolling(14).mean()
    #df["same_day_flip_1stdev_roi_30d_ma"] = df["same_day_flip_1stdev_roi"].rolling(30).mean()
    #df[f"same_day_flip_1stdev_roi_{moving_average_window_size}d_ma"] = df["same_day_flip_1stdev_roi"].rolling(moving_average_window_size).mean()
    #df["same_day_flip_2stdev_roi_30d_ma"] = df["same_day_flip_2stdev_roi"].rolling(30).mean()

    #df["10%_sell_sold"] = df["sell_sold"] * 0.1
    #df["10%_sell_sold_7d_ma"] = df["10%_sell_sold"].rolling(7).mean()
    #df["10%_sell_sold_14d_ma"] = df["10%_sell_sold"].rolling(14).mean()
    #df[f"10%_sell_sold_{moving_average_window_size}d_ma"] = df["10%_sell_sold"].rolling(moving_average_window_size).mean()
    #df["10%_sell_sold_30d_stdev"] = df["10%_sell_sold"].rolling(30).std()
    #df["10%_sell_sold_30d_ma+2stdev"] = df["10%_sell_sold_30d_ma"] + (2 * df["10%_sell_sold_30d_stdev"])
    #df["10%_sell_sold_30d_ma-2stdev"] = df["10%_sell_sold_30d_ma"] - (2 * df["10%_sell_sold_30d_stdev"])

    #df["10%_sell_value"] = df["sell_value"] * 0.1
    #df["10%_sell_value_30d_ma"] = df["10%_sell_value"].rolling(30).mean()
    #df["10%_sell_value_30d_stdev"] = df["10%_sell_value"].rolling(30).std()
    #df["10%_sell_value_30d_ma+2stdev"] = df["10%_sell_value_30d_ma"] + (2 * df["10%_sell_value_30d_stdev"])
    #df["10%_sell_value_30d_ma-2stdev"] = df["10%_sell_value_30d_ma"] - (2 * df["10%_sell_value_30d_stdev"])

    #df["roi_value_on_10%_sell_value"] = (df["10%_sell_value"] * df["same_day_flip_roi"]) - df["10%_sell_value"]
    #df["roi_value_on_10%_sell_value_30d_rolling_sum"] = df["roi_value_on_10%_sell_value"].rolling(30).sum()
    #df["roi_value_on_10%_sell_value_30d_rolling_std"] = df["roi_value_on_10%_sell_value"].rolling(30).std()

    # Expected total profit if you sold 10% of volume every day for 30 days
    #df["roi_value_on_10%_sell_value_30d_rolling_sum_30d_ma"] = df["roi_value_on_10%_sell_value_30d_rolling_sum"].rolling(30).mean()

    return Analysis(item, df[[
        #"buy_sold",
        f"buy_sold_{moving_average_window_size}d_ma",
        #"buy_value",
        #"buy_value_30d_ma",
        #"buy_price_avg",
        #"buy_price_avg_-1stdev",
        f"buy_price_avg_-1stdev_{moving_average_window_size}d_ma",
        #"buy_price_avg_-2stdev",

        #"sell_sold",
        #"sell_sold_7d_ma",
        #"sell_sold_14d_ma",
        f"sell_sold_{moving_average_window_size}d_ma",
        #"sell_value",
        #"sell_value_30d_ma",
        #"sell_price_min",
        #"sell_price_avg",
        #"sell_price_avg_+1stdev",
        f"sell_price_avg_+1stdev_{moving_average_window_size}d_ma",
        #"sell_price_avg_+2stdev",

        #"buy_price_avg_-2stdev",
        #"sell_price_avg_+2stdev",

        #"same_day_flip_profit",
        #"same_day_flip_roi",
        #"same_day_flip_roi_30d_ma",
        "same_day_flip_1stdev_roi",
        #"same_day_flip_1stdev_roi_14d_ma",
        #f"same_day_flip_1stdev_roi_{moving_average_window_size}d_ma",
        #"same_day_flip_2stdev_roi_30d_ma",

        #"10%_sell_sold",
        #"10%_sell_sold_30d_ma-2stdev",
        #"10%_sell_sold_7d_ma",
        #"10%_sell_sold_14d_ma",
        #f"10%_sell_sold_{moving_average_window_size}d_ma",
        #"10%_sell_sold_30d_ma+2stdev",
        #"10%_sell_value",
        #"10%_sell_value_30d_ma-2stdev",
        #"10%_sell_value_30d_ma",
        #"10%_sell_value_30d_ma+2stdev",
        #"roi_value_on_10%_sell_value",
        #"roi_value_on_10%_sell_value_30d_rolling_sum",
        #"roi_value_on_10%_sell_value_30d_rolling_std",
        #"roi_value_on_10%_sell_value_30d_rolling_sum_30d_ma",
        #f"sell_price_avg_pct_change_{moving_average_window_size}d_ma",
    ]], outlier_count=buy_outlier_count+sell_outlier_count)

def analysis_to_daily_flip_report(analysis: Analysis) -> DailyFlipReport:
    """TODO"""

    gw2bltc_url = f"https://www.gw2bltc.com/en/item/{analysis.item.id}"
    buy_volume, buy_price, sell_volume, sell_price, roi = analysis.df.iloc[-1]
    max_buy_count = min((buy_volume // 10), (sell_volume // 10)) if roi > 1.0 else 0

    return DailyFlipReport(
        gw2bltc_url=gw2bltc_url,
        item_id=analysis.item.id,
        item_name=analysis.item.name,
        return_on_investment=roi,
        sell_volume=sell_volume,
        buy_volume=buy_volume,
        max_buy_count=max_buy_count,
        buy_price=Coins(buy_price),
        max_invest=Coins(buy_price * max_buy_count),
        sell_price=Coins(sell_price),
        outlier_count=analysis.outlier_count,
    )

def filter_flips(flips: pd.DataFrame, min_sell_volume: int = 0, min_buy_volume: int = 0, min_buy_count: int = 0, min_buy_price: Coins = Coins()) -> pd.DataFrame:
    """Remvoe unprofitable flips."""
    
    def remove_rows_lt(df: pd.DataFrame, column_name: str, min_quantity: float) -> pd.DataFrame:
        """Remove rows for which df.column_name < min_quantity is true."""

        if df[df[column_name] < min_quantity].empty:
            return df
        items_to_be_removed = df[df[column_name] < min_quantity]["item_name"]

        logger.debug(f"Removed {len(items_to_be_removed)} items ({column_name} < {min_quantity}): {', '.join(items_to_be_removed)}")

        return df[df[column_name] >= min_quantity]

    flips = remove_rows_lt(flips, "return_on_investment", 1)
    flips = remove_rows_lt(flips, "sell_volume", min_sell_volume)
    flips = remove_rows_lt(flips, "buy_volume", min_buy_volume)
    flips = remove_rows_lt(flips, "max_buy_count", min_buy_count)
    flips = remove_rows_lt(flips, "buy_price", min_buy_price)

    return flips

def sort_and_format_flips(flips: pd.DataFrame) -> pd.DataFrame:
    """Find profitable flips for ITEMS."""

    # Sort by return on investment (ROI)
    flips.sort_values(by="return_on_investment", ascending=False, inplace=True)
    flips.reset_index(drop=True, inplace=True)

    flips["roi"] = flips["return_on_investment"] - 1
    flips["buy_stacks"] = (flips["max_buy_count"] / stack_size).apply(math.floor)
    flips["buy_stack_price"] = flips["buy_price"] * stack_size
    flips["total_buy_price"] = flips["buy_price"] * stack_size * flips["buy_stacks"]
    flips["sell_stack_price"] = flips["sell_price"] * stack_size
    flips["invest"] = flips["buy_price"] * flips["buy_stacks"] * stack_size
    flips["invest_cum_sum"] = flips["invest"].cumsum()

    return flips
"""
def pretty_print_flip(flip: pf.DataFrame) -> None:

    print(f"Found {df.shape[0]} profitable flips.")
    display(Markdown('---'))
    
    for index, row in flip[[
        "item_id",
        "item_name",
        "gw2bltc_url",
        "roi",
        "buy_stacks",
        "buy_price",
        "total_buy_price",
        "sell_price",
    ]].iterrows():
        item_id, item_name, gw2bltc_url, roi, buy_stacks, buy_price, total_buy_price, sell_price = row
        full_df = history_to_pandas(entries[item_id])[-90:][[
            "utc_timestamp",
            "sell_price_avg",
            "buy_price_avg",
            "sell_sold",
            "buy_sold",
            "sell_listed",
            "buy_listed",
        ]]
        fig = make_subplots(rows=3, cols=1,shared_xaxes=True, vertical_spacing=0.02)
        fig.add_trace(plotly.graph_objects.Scatter(x=full_df["utc_timestamp"], y=full_df["sell_price_avg"], name="Sell price"), row=1, col=1)
        fig.add_trace(plotly.graph_objects.Scatter(x=full_df["utc_timestamp"], y=full_df["buy_price_avg"], name="Buy price"), row=1, col=1)
        fig.add_trace(plotly.graph_objects.Scatter(x=full_df["utc_timestamp"], y=full_df["sell_listed"], name="Supply"), row=2, col=1)
        fig.add_trace(plotly.graph_objects.Scatter(x=full_df["utc_timestamp"], y=full_df["buy_listed"], name="Demand"), row=2, col=1)
        fig.add_trace(plotly.graph_objects.Bar(x=full_df["utc_timestamp"], y=full_df["sell_sold"], name="Sell volume"), row=3, col=1)
        fig.add_trace(plotly.graph_objects.Bar(x=full_df["utc_timestamp"], y=full_df["buy_sold"], name="Buy volume"), row=3, col=1)
        fig.update_layout(title_text=f"{item_name} (+{'{:,.2%}'.format(roi)}, {total_buy_price})", hovermode='x')
        fig.show()

        print(item_name)
        print(f"Buy {buy_stacks} stacks")
        print(f"Buy price:  {buy_price}")
        print(f"Sell price: {sell_price}")
        print(f"Total buy price: {total_buy_price}")
        
        display(Markdown('---'))
"""
        
def pretty_print_flip(entries: pd.DataFrame, item_id: int, item_name: str, roi: float, buy_stacks: int, buy_price: Coins, total_buy_price: Coins, sell_price: Coins) -> None:
    """Print a flipping buy/sell plan for ITEMS."""

    full_df = history_to_pandas(entries)[-90:][[
        "utc_timestamp",
        "sell_price_avg",
        "buy_price_avg",
        "sell_sold",
        "buy_sold",
        "sell_listed",
        "buy_listed",
    ]]
    fig = make_subplots(rows=3, cols=1,shared_xaxes=True, vertical_spacing=0.02)
    fig.add_trace(plotly.graph_objects.Scatter(x=full_df["utc_timestamp"], y=full_df["sell_price_avg"], name="Sell price"), row=1, col=1)
    fig.add_trace(plotly.graph_objects.Scatter(x=full_df["utc_timestamp"], y=full_df["buy_price_avg"], name="Buy price"), row=1, col=1)
    fig.add_trace(plotly.graph_objects.Scatter(x=full_df["utc_timestamp"], y=full_df["sell_listed"], name="Supply"), row=2, col=1)
    fig.add_trace(plotly.graph_objects.Scatter(x=full_df["utc_timestamp"], y=full_df["buy_listed"], name="Demand"), row=2, col=1)
    fig.add_trace(plotly.graph_objects.Bar(x=full_df["utc_timestamp"], y=full_df["sell_sold"], name="Sell volume"), row=3, col=1)
    fig.add_trace(plotly.graph_objects.Bar(x=full_df["utc_timestamp"], y=full_df["buy_sold"], name="Buy volume"), row=3, col=1)
    fig.update_layout(title_text=f"{item_name} (+{'{:,.2%}'.format(roi)}, {total_buy_price})", hovermode='x')
    fig.show()

    print(item_name)
    print(f"Buy {buy_stacks} stacks")
    print(f"Buy price:  {buy_price}")
    print(f"Sell price: {sell_price}")
    print(f"Total buy price: {total_buy_price}")


In [ ]:
logger.debug("---")
moving_average_window_size = 14

db = Gw2TpDb(database_path="gw2trader.sqlite", auto_update=True)
#db.populate_items()

top_1000_sold_items = get_top_1000_sold_items()
if top_1000_sold_items is None:
    print("Couldn't find 1000 top-sold items")
    quit()
items = top_1000_sold_items

"""
For debugging:

items=[
    Item(id=82991, name="Bag of Coffee Beans"),
    Item(id=19727, name="Seasoned Wood Log"),
    Item(id=43363, name="Dragonfish Candy"),
]
"""

entries = db.get_dailies(list(map(lambda item: item.id, items)))
if entries is None:
    logger.debug(f"History entries empty")
    quit()
all_flips = pd.DataFrame([report.__dict__ for report in map(analysis_to_daily_flip_report, map(lambda item: analyze_daily_flip(item, entries[item.id][moving_average_window_size*2*-1:], moving_average_window_size), items))])
good_flips = sort_and_format_flips(filter_flips(
    all_flips,
    min_buy_count=stack_size,
    min_buy_price=Coins(5),
    # Use stack_size*10 so our min 10%-of-volume is likely to be >1 stack
    min_buy_volume=stack_size*10,
    min_sell_volume=stack_size*10))
if good_flips.empty:
    print("Preconditions eliminated all candidate items. No profitable flips.")
    quit()

print(f"Found {good_flips.shape[0]} profitable flips.")

In [ ]:
flip_dropdown = widgets.Dropdown(
    options=[f"{flip.item_name} (+{'{:,.2%}'.format(flip.roi)}, {flip.total_buy_price})" for index, flip in good_flips.iterrows()],
    description="Flip:",
    disabled=False,
    layout={"width": "max-content"},
)
dropdown_output = widgets.Output()
flip_output = widgets.Output()

display(flip_dropdown, dropdown_output)
display(flip_output)
with flip_output:
    flip = list(good_flips.iterrows())[0][1]
    pretty_print_flip(entries[flip.item_id], flip.item_id, flip.item_name, flip.roi, flip.buy_stacks, flip.buy_price, flip.total_buy_price, flip.sell_price)

def on_value_change(change):
    with flip_output:
        flip_output.clear_output()
        flip = list(good_flips.iterrows())[change.owner.index][1]
        pretty_print_flip(entries[flip.item_id], flip.item_id, flip.item_name, flip.roi, flip.buy_stacks, flip.buy_price, flip.total_buy_price, flip.sell_price)

flip_dropdown.observe(on_value_change, names='value')